# Exercícios do Minicurso

## Quadro-resumo dos exercícios

| Notebook               | Título do Exercício                                    | Objetivo Didático                                                                 |
|------------------------|--------------------------------------------------------|-----------------------------------------------------------------------------------|
| `basico.ipynb`         | Municípios prioritários para políticas públicas        | Filtrar e ordenar municípios com base em renda e número de famílias               |
| `joins.ipynb`          | Infos Municípios prioritários para políticas públicas  | Adicionar as informações do município                                             |
| `with_clause.ipynb`    | Mapeamento de famílias vulneráveis por município       | Cruzar tabelas e agregar famílias por município e perfil de instrução e renda     |
| `window_functions.ipynb` | Desigualdade de renda dentro das regiões             | Usar funções de janela para classificar as regiões por desilgualdade              |

---

## Detalhamento dos exercícios

### `basico.ipynb` – Municípios prioritários para políticas públicas

Liste os **10 municípios** onde:
- a **renda média familiar** é inferior a R$ 200, e  
- há mais de 50 famílias cadastradas.

Mostre:
- `cd_ibge`, `renda_media`, `total_familias`  
Ordene do menor para o maior valor médio de renda.

---

### `joins.ipynb` – Infos Municípios prioritários para políticas públicas

Liste os **10 municípios** onde:
- a **renda média familiar** é inferior a R$ 200, e  
- há mais de 50 famílias cadastradas.
- utilize a tabela de informações municípios do IBGE

Mostre:
- `municipio`, `uf`, `regiao`, `total_familias_em_risco`  
Ordene do menor para o maior valor médio de renda.

---

### `with_clause.ipynb` – Mapeamento de famílias vulneráveis por município

Liste os **10 municípios** com o maior número de famílias que:
- possuem **renda média familiar inferior a R$ 200**, e  
- têm **ao menos uma pessoa sem instrução formal** (`cod_curso_frequentou_pessoa_memb`).

Mostre:
- `municipio`, `uf`, `regiao`, `total_familias`  
Ordene pelo maior número de famílias em risco.

---

### `window_functions.ipynb` – Desigualdade de renda dentro das regiões

Calcule a média de renda das famílias por região
- 10% que recebem menos
- 10% que recebem mais
- diferença entre as médias

Mostre:
- `regiao`, `b_10`, `t_10`, `desigualdade`  
Ordene pela maior desigualdade.


# Resolução

In [1]:
from common import DatabaseConnection

conn = DatabaseConnection("database.duckdb")

2025-06-27 23:21:50,495 - INFO - Connected to database: database.duckdb


In [2]:
conn.execute("""
    SELECT
        cd_ibge, 
        AVG(vlr_renda_media_fam) AS renda_media,
        COUNT(*) AS total_familias
    FROM familias
    GROUP BY cd_ibge
    HAVING total_familias > 50 AND renda_media < 200
    ORDER BY renda_media ASC
    LIMIT 10;
""")

2025-06-27 23:22:08,418 - INFO - Executing query...


2025-06-27 23:22:09,103 - INFO - Query executed successfully


,cd_ibge,renda_media,total_familias
0,2106409,35.330769,260
1,2201770,36.900000,220
2,2111672,37.950000,100
3,1600550,41.648649,111
4,2701506,43.157895,76
5,2100808,46.441860,258
6,1600105,48.196774,310
7,1600253,50.399293,283
8,2100709,52.162523,1649
9,1506401,54.283019,53


In [11]:
conn.execute("""
    SELECT
        *
    FROM ibge_municipios
    LIMIT 10;
""")

2025-06-26 23:57:18,611 - INFO - Executing query...
2025-06-26 23:57:18,620 - INFO - Query executed successfully


,municipio-id,municipio-nome,microrregiao-id,microrregiao-nome,mesorregiao-id,mesorregiao-nome,regiao-imediata-id,regiao-imediata-nome,regiao-intermediaria-id,regiao-intermediaria-nome,UF-id,UF-sigla,UF-nome,regiao-id,regiao-sigla,regiao-nome
0,1100015,Alta Floresta D'Oeste,11006,Cacoal,1102,Leste Rondoniense,110005,Cacoal,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
1,1100023,Ariquemes,11003,Ariquemes,1102,Leste Rondoniense,110002,Ariquemes,1101,Porto Velho,11,RO,Rondônia,1,N,Norte
2,1100031,Cabixi,11008,Colorado do Oeste,1102,Leste Rondoniense,110006,Vilhena,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
3,1100049,Cacoal,11006,Cacoal,1102,Leste Rondoniense,110005,Cacoal,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
4,1100056,Cerejeiras,11008,Colorado do Oeste,1102,Leste Rondoniense,110006,Vilhena,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
5,1100064,Colorado do Oeste,11008,Colorado do Oeste,1102,Leste Rondoniense,110006,Vilhena,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
6,1100072,Corumbiara,11008,Colorado do Oeste,1102,Leste Rondoniense,110006,Vilhena,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
7,1100080,Costa Marques,11002,Guajará-Mirim,1101,Madeira-Guaporé,110004,Ji-Paraná,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
8,1100098,Espigão D'Oeste,11006,Cacoal,1102,Leste Rondoniense,110005,Cacoal,1102,Ji-Paraná,11,RO,Rondônia,1,N,Norte
9,1100106,Guajará-Mirim,11002,Guajará-Mirim,1101,Madeira-Guaporé,110001,Porto Velho,1101,Porto Velho,11,RO,Rondônia,1,N,Norte


In [24]:
conn.execute("""
SELECT COUNT(*) AS total_familias_em_risco FROM familias GROUP BY cd_ibge
""")

2025-06-28 00:00:11,991 - INFO - Executing query...
2025-06-28 00:00:12,022 - INFO - Query executed successfully


,total_familias_em_risco
0,392
1,66
2,319
3,1634
4,381
...,...
5529,4215
5530,15067
5531,16261
5532,22712


In [ ]:
conn.execute("""
    SELECT
        "municipio-nome" AS municipio,
        "UF-sigla" AS UF,
        "regiao-imediata-nome" AS regiao,
        COUNT(familias.*) AS total_familias_em_risco
    FROM ibge_municipios, familias
    WHERE "municipio-id" IN (SELECT cd_ibge FROM familias GROUP BY cd_ibge 
    HAVING (AVG(vlr_renda_media_fam) < 200 AND COUNT(*) > 50))
    LIMIT 10
""")

2025-06-28 00:12:29,776 - INFO - Executing query...
2025-06-28 00:12:29,847 - INFO - Query executed successfully


,municipio,UF,regiao
0,Assis Brasil,AC,Brasiléia
1,Assis Brasil,AC,Brasiléia
2,Assis Brasil,AC,Brasiléia
3,Assis Brasil,AC,Brasiléia
4,Assis Brasil,AC,Brasiléia
5,Assis Brasil,AC,Brasiléia
6,Assis Brasil,AC,Brasiléia
7,Assis Brasil,AC,Brasiléia
8,Assis Brasil,AC,Brasiléia
9,Assis Brasil,AC,Brasiléia
